In [34]:
from pytrends.request import TrendReq
import datetime

# Login to Google. Only need to run this once, the rest of requests will use the same session.
pytrend = TrendReq()

In [35]:
def getWeeks():

    dStart = input("Enter date of start epidemiological week in DD-MM-YYYY format: ")
    dStart = [int(i) for i in dStart.split("-")]
    start = datetime.datetime(dStart[2],dStart[1],dStart[0])
    
    dEnd = input("Enter date of end epidemiological week in DD-MM-YYYY format: ")
    dEnd = [int(i) for i in dEnd.split("-")]
    end = datetime.datetime(dEnd[2],dEnd[1],dEnd[0])
    
    nextWeek = getNextWeek(start)
    weeks = [start]
    
    while nextWeek <= end:
        if(nextWeek < datetime.datetime.today()):
            weeks.append(nextWeek)
            nextWeek = getNextWeek(nextWeek)
    
    return weeks

In [36]:
def getNextWeek(timeframe):
    return timeframe+datetime.timedelta(days=7)

In [37]:
def getYMD(timeframe):
    return "{}/{}/{}".format(timeframe.day,timeframe.month,timeframe.year)

In [38]:
year = input("Enter year: ")

Enter year: 2017


In [39]:
countryCode = input("Enter country code: ")

Enter country code: CO


In [40]:
weeks = getWeeks()

Enter date of start epidemiological week in DD-MM-YYYY format: 01-01-2017
Enter date of end epidemiological week in DD-MM-YYYY format: 30-12-2017


In [42]:
with open("{}/google/{}.csv".format(year, countryCode), "w") as file:
    weekEpiArray = []
    cities = {}
    file.write("CITY,")
    file.write(",".join(map(getYMD, weeks))+"\n")
    for i in range(len(weeks)):
        week = weeks[i]
        nextWeek = getNextWeek(week)
        year = week.year
        month = week.month
        day = week.day
        nextWeekC = nextWeek - datetime.timedelta(days=1)
        timeframe = "{}-{}-{} {}-{}-{}".format(year,month,day,nextWeekC.year,nextWeekC.month,nextWeekC.day)
        print("{}-{}-{} -> {}-{}-{}".format(year,month,day,nextWeekC.year,nextWeekC.month,nextWeekC.day))
        pytrend.build_payload(kw_list=['zika'], geo=countryCode, timeframe=timeframe)
        results = pytrend.interest_by_region(resolution="CITY")
        # if a city's search index is 0 it doesn't appear on results, so we have to keep track of it
        tempWeekArray = {}
        for index, row in results.iterrows():
            tempWeekArray[index] = row["zika"]
            if index not in cities:
                cities[index] = index #Store in dictionary to get O(1) access
        weekEpiArray.append(tempWeekArray)
        print("DONE", week)
        
    # Iterate on weekEpiArray and replace non existent cities with zeroes
    searchesByCity = {}
    for city in cities:
        searchesByCity[city] = []
        for week in weekEpiArray:
            if city not in week:
                searchesByCity[city].append(0)
            else:
                searchesByCity[city].append(week[city])
                
    # Finally write to file
    for city in sorted(searchesByCity.keys()):
        file.write("{},".format(city))
        file.write(",".join(map(str,searchesByCity[city])))
        file.write("\n")

2017-1-1 -> 2017-1-7
DONE 2017-01-01 00:00:00
2017-1-8 -> 2017-1-14
DONE 2017-01-08 00:00:00
2017-1-15 -> 2017-1-21
DONE 2017-01-15 00:00:00
2017-1-22 -> 2017-1-28
DONE 2017-01-22 00:00:00
2017-1-29 -> 2017-2-4
DONE 2017-01-29 00:00:00
2017-2-5 -> 2017-2-11
DONE 2017-02-05 00:00:00
2017-2-12 -> 2017-2-18
DONE 2017-02-12 00:00:00
2017-2-19 -> 2017-2-25
DONE 2017-02-19 00:00:00
2017-2-26 -> 2017-3-4
DONE 2017-02-26 00:00:00
2017-3-5 -> 2017-3-11
DONE 2017-03-05 00:00:00
2017-3-12 -> 2017-3-18
DONE 2017-03-12 00:00:00
2017-3-19 -> 2017-3-25
DONE 2017-03-19 00:00:00
2017-3-26 -> 2017-4-1
DONE 2017-03-26 00:00:00
2017-4-2 -> 2017-4-8
DONE 2017-04-02 00:00:00
2017-4-9 -> 2017-4-15
DONE 2017-04-09 00:00:00
2017-4-16 -> 2017-4-22
DONE 2017-04-16 00:00:00
2017-4-23 -> 2017-4-29
DONE 2017-04-23 00:00:00
2017-4-30 -> 2017-5-6
DONE 2017-04-30 00:00:00
2017-5-7 -> 2017-5-13
DONE 2017-05-07 00:00:00
2017-5-14 -> 2017-5-20
DONE 2017-05-14 00:00:00
2017-5-21 -> 2017-5-27
DONE 2017-05-21 00:00:00
2017-